In [1]:
!pip install transformers datasets fasttext -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.7 MB/s eta 0:00

In [2]:
from datasets import load_dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModel, get_scheduler, AdamW
from torch.utils.data import DataLoader, RandomSampler
from torch.nn.utils.rnn import pad_sequence
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm.auto import tqdm
from torch import optim
import numpy as np
from tqdm import tqdm
import fasttext

In [3]:
STS = load_dataset("stsb_multi_mt", "es")

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Dataset stsb_multi_mt downloaded and prepared to /root/.cache/huggingface/datasets/stsb_multi_mt/es/1.0.0/a5d260e4b7aa82d1ab7379523a005a366d9b124c76a5a5cf0c4c5365458b0ba9. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
pd.DataFrame(STS["train"]).sample(5)

,sentence1,sentence2,similarity_score
495,Un hombre y una mujer se están besando.,Un hombre y una mujer están hablando entre ellos.,0.60
4945,Obama promete reavivar la economía,Obama presionará al Congreso para que actúe so...,2.60
1145,Una sala de estar con una mesa de comedor al f...,Una sala de estar con grandes sofás y una mesa...,3.60
2926,El cierre del programa afectó las ganancias po...,La compañía dijo que esto afectaba a las ganan...,3.25
4251,La fiebre del oro británica silencia las queja...,2 soldados británicos muertos en un ataque int...,0.60


In [5]:
!pip install transformers -q

In [6]:
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [7]:
tokenizer.model_max_length = 100
PAD_IDX = tokenizer.pad_token_id

def tokenize_sentence1(example):
    sentence1_tok = tokenizer(example["sentence1"], truncation=True, padding=False)
    sentence1_tok['sentence1_ids'] = sentence1_tok.pop('input_ids')
    return sentence1_tok

def tokenize_sentence2(example):
    sentence2_tok = tokenizer(example["sentence2"], truncation=True, padding=False)
    sentence2_tok['sentence2_ids'] = sentence2_tok.pop('input_ids')
    return sentence2_tok

tokenized_STS = STS.map(tokenize_sentence1, batched=True)
tokenized_STS = tokenized_STS.map(tokenize_sentence2, batched=True)

Map:   0%|          | 0/5749 [00:00<?, ? examples/s]

Map:   0%|          | 0/1379 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/5749 [00:00<?, ? examples/s]

Map:   0%|          | 0/1379 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [8]:
def generate_attention_mask(tensor):
    attention_mask = (tensor != PAD_IDX).float()
    return attention_mask

In [9]:
def collate_batch(batch):
    s1      = [torch.LongTensor(example['sentence1_ids'])\
                                              for example in batch]
    s2 = [torch.LongTensor(example['sentence2_ids'])\
                                                for example in batch]
    s1_ids   = pad_sequence(s1, batch_first=True, padding_value=PAD_IDX)
    s2_ids   = pad_sequence(s2, batch_first=True, padding_value=PAD_IDX)
    #att masks
    s1_ids_att_mask = generate_attention_mask(s1_ids)
    s2_ids_att_mask = generate_attention_mask(s2_ids)

    return ([example['sentence1'] for example in batch],
           [example['sentence2'] for example in batch],
           s1_ids, s2_ids, torch.tensor([example['similarity_score'] \
                                                        for example in batch]),
          s1_ids_att_mask,
          s2_ids_att_mask)

test_dataloader = DataLoader(tokenized_STS["test"], batch_size=16, collate_fn=collate_batch)

In [10]:
class SROBERTA(nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.base_model = base_model
        self.cos = nn.CosineSimilarity(dim=1, eps=1e-6)

    def forward(self, sentenceA, sentenceB, att_A, att_B):
        last_hidden_state_A = self.base_model(sentenceA)[0]
        last_hidden_state_B = self.base_model(sentenceB)[0]

        pooled_output_A = torch.mean(torch.matmul(att_A, last_hidden_state_A), dim=1)
        pooled_output_B = torch.mean(torch.matmul(att_B, last_hidden_state_B), dim=1)
        # print(f"pooled_outputA: {pooled_output_A}\n pooled_outputA.shape: {pooled_output_A.shape}")
        out = self.cos(pooled_output_A, pooled_output_B)
        return out

In [11]:
def _get_ranks(x: torch.Tensor, device) -> torch.Tensor:
    tmp = x.argsort()
    ranks = torch.zeros_like(tmp, device=device)
    ranks[tmp] = torch.arange(len(x), device=device)
    return ranks

def spearman_correlation(x: torch.Tensor, y: torch.Tensor, device):
    """Compute correlation between 2 1-D vectors
    Args:
        x: Shape (N, )
        y: Shape (N, )
    """
    x_rank = _get_ranks(x, device)
    y_rank = _get_ranks(y, device)

    n = x.size(0)
    upper = 6 * torch.sum((x_rank - y_rank).pow(2))
    down = n * (n ** 2 - 1.0)
    return 1.0 - (upper / down)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ROBERTA_BASE=AutoModel.from_pretrained("xlm-roberta-base")
SROBERTA_FINETUNNED = AutoModel.from_pretrained("Csuarezg/SBERTA-finetuned", revision = "fa57675")

modelos = [ROBERTA_BASE, SROBERTA_FINETUNNED]
spearman_corr = []
for modelo in modelos:
  model = SROBERTA(modelo)
  model = model.to(device)

  optimizer = AdamW(model.parameters(), lr=2e-5)
  corr_batch = []


  for batch in tqdm(test_dataloader):
      _, _, batch_sent1, batch_sent2, batch_labels, batch_sent1_att, batch_sent2_att = batch
      batch_sent1 = batch_sent1.to(device)
      batch_sent2 = batch_sent2.to(device)
      batch_sent1_att = batch_sent1_att.to(device)
      batch_sent2_att = batch_sent2_att.to(device)
      batch_labels = batch_labels.to(device)

      preds = model(batch_sent1, batch_sent2, batch_sent1_att, batch_sent2_att)
      sp_corr = spearman_correlation(preds, batch_labels, device)
      corr_batch.append(sp_corr.cpu())
  spearman_corr.append(np.mean(corr_batch))

  print(f"\nSPEARMAN CORRELATION :{np.mean(corr_batch)}")

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 87/87 [00:07<00:00, 10.98it/s]



SPEARMAN CORRELATION :0.1951318085193634


100%|██████████| 87/87 [00:04<00:00, 19.39it/s]


SPEARMAN CORRELATION :0.5023664236068726


In [13]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.es.300.bin.gz && gunzip cc.es.300.bin.gz

--2023-06-15 05:33:34--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.es.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.224.2.6, 13.224.2.88, 13.224.2.21, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.224.2.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4500107671 (4.2G) [application/octet-stream]
Saving to: ‘cc.es.300.bin.gz’

cc.es.300.bin.gz    100%[===================>]   4.19G   244MB/s    in 23s     

2023-06-15 05:33:57 (184 MB/s) - ‘cc.es.300.bin.gz’ saved [4500107671/4500107671]



In [14]:
model_fasttext = fasttext.load_model('cc.es.300.bin')

In [15]:
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
corr_batch = []
for batch in tqdm(test_dataloader):
    batch_sent1_text, batch_sent2_text, _, _, batch_labels, _, _ = batch
    sentences1 = [model_fasttext.get_sentence_vector(x) for x in batch_sent1_text]
    sentences2 = [model_fasttext.get_sentence_vector(x) for x in batch_sent2_text]
    preds = cos(
      torch.tensor(sentences1),
      torch.tensor(sentences2)
    )
    corr_batch.append(
        spearman_correlation(preds, batch_labels, 'cpu')
    )
spearman_corr.append(np.mean(corr_batch))
print(f"\nSPEARMAN CORRELATION :{np.mean(corr_batch)}")

  0%|          | 0/87 [00:00<?, ?it/s]<ipython-input-15-679547faeb4e>:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  torch.tensor(sentences1),
100%|██████████| 87/87 [00:00<00:00, 115.60it/s]


SPEARMAN CORRELATION :0.4206896424293518


In [16]:
index = ['SPEARMAN CORRELATION VANILLA ROBERTA',
         'SPEARMAN CORRELATION SBERTA FINETUNNED',
         'SPEARMAN CORRELATION FASTTEXT']

values = [spearman_corr[0], spearman_corr[1], spearman_corr[2]]

df = pd.DataFrame(values, index=index, columns=['SPEARMAN CORRELATION IN STS TASK'])
df

,SPEARMAN CORRELATION IN STS TASK
SPEARMAN CORRELATION VANILLA ROBERTA,0.195132
SPEARMAN CORRELATION SBERTA FINETUNNED,0.502366
SPEARMAN CORRELATION FASTTEXT,0.420690
